In [37]:
from __future__ import print_function
import numpy as np
import pandas as pd
import pylab as pl
import geopandas as gpd
import shapely
from fiona.crs import from_epsg
import os
import statsmodels as stm
import urllib2
import urllib
import StringIO
import ast
from datetime import datetime, timedelta
import time
import io
import json
import requests
import shutil
import datetime as dt
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from sklearn import preprocessing
import statsmodels.formula.api as smf
import pytz

%pylab inline

Populating the interactive namespace from numpy and matplotlib


---

# WEATHER

In [13]:
# Downloadind the weather data - don't run again
#!unzip "DarkSky_Data.zip"

Archive:  DarkSky_Data.zip
  inflating: DarkSky_Data.json       


In [2]:
# Reading in the weather 

weather = pd.read_json("DarkSky_Data.json")

In [35]:
weather.head()

,apparentTemperature,cloudCover,dewPoint,humidity,icon,precipAccumulation,precipIntensity,precipProbability,precipType,pressure,summary,temperature,time,visibility,windBearing,windSpeed,datetime
0,26.80,0.82,28.43,0.85,partly-cloudy-night,NaN,0.0,0.0,NaN,1009.73,Mostly Cloudy,32.48,1136091600,8.84,335.0,6.00,2006-01-01 05:00:00
1,27.04,0.77,28.32,0.85,partly-cloudy-night,NaN,0.0,0.0,NaN,1010.35,Mostly Cloudy,32.33,1136095200,7.67,345.0,5.49,2006-01-01 06:00:00
2,27.21,0.78,28.34,0.85,partly-cloudy-night,NaN,0.0,0.0,NaN,1011.40,Mostly Cloudy,32.29,1136098800,8.69,344.0,5.23,2006-01-01 07:00:00
3,27.01,0.34,27.97,0.84,partly-cloudy-night,NaN,0.0,0.0,NaN,1012.17,Partly Cloudy,32.19,1136102400,8.79,345.0,5.32,2006-01-01 08:00:00
4,26.65,0.95,28.10,0.85,cloudy,NaN,0.0,0.0,NaN,1012.67,Overcast,32.00,1136106000,8.41,339.0,5.49,2006-01-01 09:00:00


In [40]:
# # Adding datetime column to weather data

weather['datetime'] = pd.to_datetime(weather['time'], unit='s', utc=True)

In [54]:
# Adjusting the datetime for timezone 

tz = pytz.timezone('America/New_York')

adjustDateTime = []
for i in range(len(weather)):
    adjustDateTime.append(weather['datetime'][i].tz_localize(pytz.utc).tz_convert(tz).tz_localize(None))

In [55]:
# Adding the adjusted column

weather['datetime_adj'] = adjustDateTime

In [57]:
# Reweather.head()

,apparentTemperature,cloudCover,dewPoint,humidity,icon,precipAccumulation,precipIntensity,precipProbability,precipType,pressure,summary,temperature,time,visibility,windBearing,windSpeed,datetime,datetime_adj
0,26.80,0.82,28.43,0.85,partly-cloudy-night,NaN,0.0,0.0,NaN,1009.73,Mostly Cloudy,32.48,1136091600,8.84,335.0,6.00,2006-01-01 05:00:00,2006-01-01 00:00:00
1,27.04,0.77,28.32,0.85,partly-cloudy-night,NaN,0.0,0.0,NaN,1010.35,Mostly Cloudy,32.33,1136095200,7.67,345.0,5.49,2006-01-01 06:00:00,2006-01-01 01:00:00
2,27.21,0.78,28.34,0.85,partly-cloudy-night,NaN,0.0,0.0,NaN,1011.40,Mostly Cloudy,32.29,1136098800,8.69,344.0,5.23,2006-01-01 07:00:00,2006-01-01 02:00:00
3,27.01,0.34,27.97,0.84,partly-cloudy-night,NaN,0.0,0.0,NaN,1012.17,Partly Cloudy,32.19,1136102400,8.79,345.0,5.32,2006-01-01 08:00:00,2006-01-01 03:00:00
4,26.65,0.95,28.10,0.85,cloudy,NaN,0.0,0.0,NaN,1012.67,Overcast,32.00,1136106000,8.41,339.0,5.49,2006-01-01 09:00:00,2006-01-01 04:00:00


In [74]:
# Removing duplicated rows

weather = weather.drop_duplicates(subset=['datetime_adj'])

---

# CRIME

In [8]:
## Downloading the crime data -- very large, don't run again

#!wget "https://data.cityofnewyork.us/api/views/qgea-i56i/rows.csv?accessType=DOWNLOAD"

--2018-02-12 22:32:08--  https://data.cityofnewyork.us/api/views/qgea-i56i/rows.csv?accessType=DOWNLOAD
Resolving data.cityofnewyork.us... 52.206.140.205
Connecting to data.cityofnewyork.us|52.206.140.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: “rows.csv?accessType=DOWNLOAD”

    [               <=>                   ] 1,427,270,109 2.71M/s   in 10m 33s 

Last-modified header invalid -- time-stamp ignored.
2018-02-12 22:42:43 (2.15 MB/s) - “rows.csv?accessType=DOWNLOAD” saved [1427270109]



In [6]:
# Reading in the crime data

crimeData = pd.read_csv("rows.csv?accessType=DOWNLOAD")

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
# Looking through the columns

crimeData.columns

Index([u'CMPLNT_NUM', u'CMPLNT_FR_DT', u'CMPLNT_FR_TM', u'CMPLNT_TO_DT',
       u'CMPLNT_TO_TM', u'RPT_DT', u'KY_CD', u'OFNS_DESC', u'PD_CD',
       u'PD_DESC', u'CRM_ATPT_CPTD_CD', u'LAW_CAT_CD', u'JURIS_DESC',
       u'BORO_NM', u'ADDR_PCT_CD', u'LOC_OF_OCCUR_DESC', u'PREM_TYP_DESC',
       u'PARKS_NM', u'HADEVELOPT', u'X_COORD_CD', u'Y_COORD_CD', u'Latitude',
       u'Longitude', u'Lat_Lon'],
      dtype='object')

In [8]:
crimeData.tail()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,...,ADDR_PCT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,PARKS_NM,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
5580030,582350583,01/01/2015,03:50:00,01/01/2016,04:00:00,01/01/2016,105,ROBBERY,399.0,"ROBBERY,COMMERCIAL UNCLASSIFIED",...,30.0,INSIDE,HOTEL/MOTEL,NaN,NaN,998372.0,240146.0,40.825818,-73.948975,"(40.825817778, -73.948974825)"
5580031,258046495,01/01/2015,01:25:00,01/01/2016,01:30:00,01/01/2016,578,HARRASSMENT 2,638.0,"HARASSMENT,SUBD 3,4,5",...,41.0,NaN,TRANSIT - NYC SUBWAY,NaN,NaN,1014468.0,238156.0,40.820315,-73.890825,"(40.820315396, -73.890824603)"
5580032,640212578,01/01/2015,00:30:00,01/01/2016,00:40:00,01/01/2016,106,FELONY ASSAULT,109.0,"ASSAULT 2,1,UNCLASSIFIED",...,109.0,FRONT OF,BAR/NIGHT CLUB,NaN,NaN,1030529.0,214093.0,40.754199,-73.832963,"(40.754199468, -73.832962523)"
5580033,496500431,06/30/2014,12:00:00,12/29/2015,12:00:00,01/01/2016,361,OFF. AGNST PUB ORD SENSBLTY &,639.0,AGGRAVATED HARASSMENT 2,...,50.0,INSIDE,RESIDENCE - APT. HOUSE,NaN,NaN,1009735.0,261272.0,40.883777,-73.907837,"(40.883776851, -73.907836928)"
5580034,256379572,12/31/2001,16:00:00,01/01/2016,10:50:00,01/01/2016,107,BURGLARY,213.0,"BURGLARY,COMMERCIAL,NIGHT",...,84.0,INSIDE,PUBLIC SCHOOL,NaN,NaN,989682.0,188334.0,40.683617,-73.980416,"(40.683616638, -73.980416007)"


In [9]:
# Filtering down to complaint dates between 01/01/2006 - 12/31/2017

crimeData['year'] = crimeData['CMPLNT_FR_DT'].str[-4:].fillna(0).astype(int)
crimeData['hour'] = crimeData['CMPLNT_FR_TM'].str[:2].fillna(0).astype(int)
crime = crimeData[((crimeData['year'] > 2005) & (crimeData['hour'] < 24))]

# Creating datetime column

crime['datetime'] = pd.to_datetime((crime['CMPLNT_FR_DT'] + ' ' + crime['CMPLNT_FR_TM']), format='%m/%d/%Y %H:%M:%S')

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [28]:
# dropping NaNs from datetime

crime = crime.dropna(axis=0, subset=['datetime'])
crime = crime.reset_index()

In [29]:
# Rounding to the nearest hour

def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +timedelta(hours=t.minute//30))

crimeRound = []

for i in range(len(crime)):
     t = crime['datetime'][i]
     crimeRound.append(hour_rounder(t)) 

In [31]:
# Adding the rounded datetime to the dataframe

crime['datetime_round'] = crimeRound

In [32]:
crime.head()

,level_0,index,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,KY_CD,OFNS_DESC,...,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,year,hour,datetime,datetime_round
0,0,0,101109527,12/31/2015,23:45:00,NaN,NaN,12/31/2015,113,FORGERY,...,NaN,1007314.0,241257.0,40.828848,-73.916661,"(40.828848333, -73.916661142)",2015,23,2015-12-31 23:45:00,2016-01-01 00:00:00
1,1,1,153401121,12/31/2015,23:36:00,NaN,NaN,12/31/2015,101,MURDER & NON-NEGL. MANSLAUGHTER,...,NaN,1043991.0,193406.0,40.697338,-73.784557,"(40.697338138, -73.784556739)",2015,23,2015-12-31 23:36:00,2016-01-01 00:00:00
2,2,2,569369778,12/31/2015,23:30:00,NaN,NaN,12/31/2015,117,DANGEROUS DRUGS,...,NaN,999463.0,231690.0,40.802607,-73.945052,"(40.802606608, -73.945051911)",2015,23,2015-12-31 23:30:00,2016-01-01 00:00:00
3,3,3,968417082,12/31/2015,23:30:00,NaN,NaN,12/31/2015,344,ASSAULT 3 & RELATED OFFENSES,...,NaN,1060183.0,177862.0,40.654549,-73.726339,"(40.654549444, -73.726338791)",2015,23,2015-12-31 23:30:00,2016-01-01 00:00:00
4,4,4,641637920,12/31/2015,23:25:00,12/31/2015,23:30:00,12/31/2015,344,ASSAULT 3 & RELATED OFFENSES,...,NaN,987606.0,208148.0,40.738002,-73.987891,"(40.7380024, -73.98789129)",2015,23,2015-12-31 23:25:00,2015-12-31 23:00:00


In [5]:
# Unique crime types

crimeData.OFNS_DESC.unique()

array(['FORGERY', 'MURDER & NON-NEGL. MANSLAUGHTER', 'DANGEROUS DRUGS',
       'ASSAULT 3 & RELATED OFFENSES', 'FELONY ASSAULT',
       'DANGEROUS WEAPONS', 'PETIT LARCENY', 'GRAND LARCENY', 'ROBBERY',
       'OFFENSES AGAINST PUBLIC ADMINI', 'CRIMINAL MISCHIEF & RELATED OF',
       'RAPE', 'INTOXICATED & IMPAIRED DRIVING', 'HARRASSMENT 2',
       'SEX CRIMES', 'BURGLARY', 'CRIMINAL TRESPASS',
       'MISCELLANEOUS PENAL LAW', 'VEHICLE AND TRAFFIC LAWS',
       'OFF. AGNST PUB ORD SENSBLTY &', 'FRAUDS',
       'GRAND LARCENY OF MOTOR VEHICLE', 'OFFENSES INVOLVING FRAUD',
       'OFFENSES AGAINST THE PERSON', 'FRAUDULENT ACCOSTING',
       'OTHER OFFENSES RELATED TO THEF', 'GAMBLING', 'ARSON',
       'POSSESSION OF STOLEN PROPERTY', 'UNAUTHORIZED USE OF A VEHICLE',
       'THEFT-FRAUD', 'DISORDERLY CONDUCT', nan, 'ADMINISTRATIVE CODE',
       'CHILD ABANDONMENT/NON SUPPORT', 'OTHER STATE LAWS (NON PENAL LA',
       'NYS LAWS-UNCLASSIFIED FELONY', "BURGLAR'S TOOLS",
       'THEFT OF SERV

---

# Merging the Data

In [78]:
df = crime.merge(weather, how='left', left_on="datetime_round", right_on="datetime_adj")

In [79]:
# Confirming that the new dataframe is the same length as the crime dataframe

print("left merge:", len(df) - len(crime))

left merge: 0


In [80]:
df.tail()

,level_0,index,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,KY_CD,OFNS_DESC,...,precipType,pressure,summary,temperature,time,visibility,windBearing,windSpeed,datetime_y,datetime_adj
5559476,5559523,5580029,593729461,01/01/2015,04:30:00,01/01/2016,04:50:00,01/01/2016,105,ROBBERY,...,NaN,1020.49,Clear,26.50,1.420106e+09,10.0,271.0,10.78,2015-01-01 10:00:00,2015-01-01 05:00:00
5559477,5559524,5580030,582350583,01/01/2015,03:50:00,01/01/2016,04:00:00,01/01/2016,105,ROBBERY,...,NaN,1020.20,Clear,26.76,1.420103e+09,10.0,258.0,11.69,2015-01-01 09:00:00,2015-01-01 04:00:00
5559478,5559525,5580031,258046495,01/01/2015,01:25:00,01/01/2016,01:30:00,01/01/2016,578,HARRASSMENT 2,...,NaN,1020.92,Clear,26.98,1.420092e+09,10.0,247.0,9.87,2015-01-01 06:00:00,2015-01-01 01:00:00
5559479,5559526,5580032,640212578,01/01/2015,00:30:00,01/01/2016,00:40:00,01/01/2016,106,FELONY ASSAULT,...,NaN,1020.92,Clear,26.98,1.420092e+09,10.0,247.0,9.87,2015-01-01 06:00:00,2015-01-01 01:00:00
5559480,5559527,5580033,496500431,06/30/2014,12:00:00,12/29/2015,12:00:00,01/01/2016,361,OFF. AGNST PUB ORD SENSBLTY &,...,NaN,1017.79,Clear,80.44,1.404144e+09,10.0,204.0,6.89,2014-06-30 16:00:00,2014-06-30 12:00:00
